# じゃんけんのリズムを作る

じゃんけんする時は、黙ってじゃんけんしてもタイミングがありませんので、みんなで掛け声をかけてじゃんけんします。また、志村けん氏が始めたと言われる、「最初にグー」
というリズムも必要です。
これをLEGOの音再生機能を使って作ります。これは、＃関数と待ちを発生させる＃関数を組み合わせて、みんなの掛け声のリズムを再現してみました。音の高さはnoteで指定しており、タイミングはtime.sleepで指定しています。気に入らない場合はいろいろ調整してください。
繰り返し実行しますので、新しい別のプログラムとして、以下を作ってください。

In [ ]:
from spike import PrimeHub
import time

prime_hub = PrimeHub()

def janken_ready1():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=77, seconds=0.2)


def janken_ready2():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=65, seconds=0.4)
    time.sleep(0.2)
    prime_hub.speaker.beep(note=65, seconds=0.1)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=77, seconds=0.2)


while True:

    # 最初はグー
    janken_ready1()
    time.sleep(0.8)

    # じゃんけんぽん
    janken_ready2()
    time.sleep(1)

リズムが決まったら先ほど作ったじゃんけんプログラムに組み込みましょう。組み込むのは音を再生する、関数janken_reeady1()と、関数janken_ready2()です。組み込んだじゃんけんプログラムは以下となります。

In [ ]:
import sys
import time
import random
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

# convert
# hand id to hand string
#  1: 'guu' , 2: 'choki' 3: 'paa'
def  hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

#
# hand1, hand2 が guu, choki, paa を選択した時、
# hand1側が勝ったのか負けたのか引き分けか？を返します
#
def judge_winner(hand1, hand2):
    if hand1 == 'guu':
        if hand2 == 'guu':
            return 'draw'
        elif hand2 == 'choki':
            return 'win'
        elif hand2 == 'paa':
            return 'lose'
    elif hand1 == 'choki':
        if hand2 == 'guu':
            return 'lose'
        elif hand2 == 'choki':
            return 'draw'
        elif hand2 == 'paa':
            return 'win'
    elif hand1 == 'paa':
        if hand2 == 'guu':
            return 'win'
        elif hand2 == 'choki':
            return 'lose'
        elif hand2 == 'paa':
            return 'draw'


prime_hub = PrimeHub()

def janken_ready1():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=77, seconds=0.2)

def janken_ready2():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=65, seconds=0.4)
    time.sleep(0.2)
    prime_hub.speaker.beep(note=65, seconds=0.1)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=77, seconds=0.2)


hand_str = None
while True:

    # 最初はぐー
    janken_ready1()
    time.sleep(0.8)

    # じゃんけんぽん
    janken_ready2()

    # get te of LEGO
    LEGO_choice = random.choice(('guu','choki','paa'))

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        hand_id = block[0][1]
        hand_str = hand_id2str(hand_id)
    result = judge_winner(hand_str, LEGO_choice)
    print(hand_str, LEGO_choice, 'you are..', result)
    time.sleep(1)

実行してみてどうですか？　じゃんけんをやっている感じが出てきましたか？　人と人とがじゃんけんをすると、勝った負けたと大騒ぎになりますね。効果音を出して場を盛り上げましょう。効果音がいろいろ用意されていますので、勝った場合は歓声が上がるようにしましょう。以下は２つの効果音を順番に再生する例です。歓声と、ビープ音です。

In [ ]:
from spike import App

app = App()
name = 'Applause 3'
play_sound(name, volume=10)

sound = 'Clown Honk 2'
app.play_sound(sound, volume=10)

# じゃんけんゲームを仕上げる

ジャンケンプログラムに組み込みます。勝ったら歓声を上げる、負けたらビープ音を鳴らすように実装してみます。

In [ ]:
import sys
import time
import random
from hub import port
from spike import App


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

# convert
# hand id to hand string
#  1: 'guu' , 2: 'choki' 3: 'paa'
def  hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

#
# hand1, hand2 が guu, choki, paa を選択した時、
# hand1側が勝ったのか負けたのか引き分けか？を返します
#
def judge_winner(hand1, hand2):
    if hand1 == 'guu':
        if hand2 == 'guu':
            return 'draw'
        elif hand2 == 'choki':
            return 'win'
        elif hand2 == 'paa':
            return 'lose'
    elif hand1 == 'choki':
        if hand2 == 'guu':
            return 'lose'
        elif hand2 == 'choki':
            return 'draw'
        elif hand2 == 'paa':
            return 'win'
    elif hand1 == 'paa':
        if hand2 == 'guu':
            return 'win'
        elif hand2 == 'choki':
            return 'lose'
        elif hand2 == 'paa':
            return 'draw'


prime_hub = PrimeHub()

def janken_ready1():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=77, seconds=0.2)

def janken_ready2():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=65, seconds=0.4)
    time.sleep(0.2)
    prime_hub.speaker.beep(note=65, seconds=0.1)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=77, seconds=0.2)


app = App()

hand_str = None
while True:

    # 最初はぐー
    janken_ready1()
    time.sleep(0.8)

    # じゃんけんぽん
    janken_ready2()

    # get te of LEGO
    LEGO_choice = random.choice(('guu','choki','paa'))

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        hand_id = block[0][1]
        hand_str = hand_id2str(hand_id)
    result = judge_winner(hand_str, LEGO_choice)
    print(hand_str, LEGO_choice, 'you are..', result)
    if result == 'win':
        name = 'Applause 3'
    elif result == 'lose':
        name = 'Clown Honk 2'
    app.play_sound(name, volume=10)
    time.sleep(1)

上記でじゃんけんプログラムは完成です。動かしてみて本当にじゃんけんをしているようになったでしょうか。さらに分かりやすくするため、LEGOが出した手をLEDマトリクスに表示させる改良を加えます（時間があって可能なら追加してください）

In [ ]:
from spike import PrimeHub
import time
hub = PrimeHub()

while True:
    hub.light_matrix.show_image('SQUARE_SMALL')
    time.sleep(1)
    hub.light_matrix.show_image('YES')
    time.sleep(1)
    hub.light_matrix.show_image('PITCHFORK')
    time.sleep(1)
    hub.light_matrix.show_image('HAPPY')
    time.sleep(1)
    hub.light_matrix.show_image('SAD')
    time.sleep(1)
    hub.light_matrix.show_image('ASLEEP')
    time.sleep(1)

以上のプログラムを全て盛り込んだソースが以下です。

In [ ]:
import sys
import time
import random
from hub import port
from spike import App
from spike import PrimeHub


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)
husky.send_CMD_REQ_ALGO(Algo.OBJECT_CLASSIFICATION)
time.sleep(1)

# convert
# hand id to hand string
#1: 'guu' , 2: 'choki' 3: 'paa'
def hand_id2str(id):
    if id == 0:
        return 'unknown'
    elif id == 1:
        return 'back'
    elif id == 2:
        return 'guu'
    elif id == 3:
        return 'choki'
    elif id == 4:
        return 'paa'

#
# hand1, hand2 が guu, choki, paa を選択した時、
# hand1側が勝ったのか負けたのか引き分けか？を返します
#
def judge_winner(hand1, hand2):
    if hand1 == 'guu':
        if hand2 == 'guu':
            return 'draw'
        elif hand2 == 'choki':
            return 'win'
        elif hand2 == 'paa':
            return 'lose'
    elif hand1 == 'choki':
        if hand2 == 'guu':
            return 'lose'
        elif hand2 == 'choki':
            return 'draw'
        elif hand2 == 'paa':
            return 'win'
    elif hand1 == 'paa':
        if hand2 == 'guu':
            return 'win'
        elif hand2 == 'choki':
            return 'lose'
        elif hand2 == 'paa':
            return 'draw'


prime_hub = PrimeHub()

def janken_ready1():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.3)
    prime_hub.speaker.beep(note=77, seconds=0.2)


def janken_ready2():
    prime_hub.speaker.beep(note=65, seconds=0.2)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=65, seconds=0.4)
    time.sleep(0.2)
    prime_hub.speaker.beep(note=65, seconds=0.1)
    time.sleep(0.1)
    prime_hub.speaker.beep(note=77, seconds=0.2)

def LEGO_show_hand(hand):
    if hand == 'guu':
        image = 'SQUARE_SMALL'
    elif hand == 'choki':
        image = 'YES'
    elif hand == 'paa':
        image = 'PITCHFORK'
    prime_hub.light_matrix.show_image(image)


app = App()

hand_str = None
while True:

    # 最初はぐー
    janken_ready1()
    LEGO_show_hand('guu')
    time.sleep(0.8)

    # じゃんけんぽん
    janken_ready2()

    # get te of LEGO
    LEGO_choice = random.choice(('guu','choki','paa'))
    LEGO_show_hand(LEGO_choice)

    # get user's te
    block = husky.read_blocks()
    print(block)
    if block and len(block) > 0:
        hand_id = block[0][1]
        hand_str = hand_id2str(hand_id)
    result = judge_winner(hand_str, LEGO_choice)
    print(hand_str, LEGO_choice, 'you are..', result)
    if result == 'win':
        name = 'Goal Cheer'
    elif result == 'lose':
        name = 'Oops'
    else:
        name = 'Bird'
    app.play_sound(name, volume=10)
    time.sleep(1)

ジャンケンゲームは以上で完成です。もし改良したいところがあればいろいろ改良してください。